In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from tabulate import tabulate

def display_dataframe(dataframe: pd.DataFrame):
    """
    Muestra una tabla de Pandas en formato tabular.

    Args:
    - dataframe (pd.DataFrame): La tabla a mostrar.
    """
    print(tabulate(dataframe, headers=dataframe.columns, tablefmt='orgtbl'))

In [ ]:
def categorize(name: str) -> str:
    """
    Categoriza el nombre de la dependencia en una de las categorías
    predefinidas.

    Args:
    - name: Nombre de la dependencia.

    Returns:
    - str: Categoría de la dependencia.
    """
    categories = {
        'PREPARATORIA': ['PREPARATORIA', 'PREPA.'],
        'FACULTAD': ['FACULTAD', 'FAC.'],
        'HOSPITAL': ['HOSPITAL'],
        'CENTRO': ['CENTRO', 'CTRO.', 'C.', 'INVESTIGAC'],
        'ADMIN': ['SECRETARÍA', 'SECRETARIA', 'SRIA.', 'DIRECCIÓN',
                    'DIRECCION', 'DEPARTAMENTO', 'DEPTO.', 'CONTRALORIA',
                    'AUDITORIA', 'TESORERIA', 'ESCOLAR', 'ABOGACÍA', 'JUNTA',
                    'RECTORIA', 'IMAGEN']
    }

    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in name:
                return category
    return 'OTRO'

def transform_date_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Transforma la columna de fecha en formato datetime.

    Args:
    - df: Tabla de Pandas.
    - column_name: Nombre de la columna de fecha.

    Returns:
    - pd.DataFrame: Tabla con la columna de fecha transformada.
    """
    df[column_name] = pd.to_datetime(df[column_name], format="%Y-%m")
    return df

def transform_into_typed_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Añade la columna 'Tipo' y 'Fecha' a la tabla.

    Args:
    - raw_df (pd.DataFrame): La tabla a transformar.

    Returns:
    - pd.DataFrame: La tabla trasnformada
    """
    raw_df["Fecha"] = pd.to_datetime(raw_df["anio"].map(str)+ "-" + raw_df["mes"].map(str), format="%Y-%m")
    raw_df["Tipo"] = raw_df["dependencia"].apply(categorize)
    trasnformed_df = raw_df.drop(columns=['anio', 'mes'])
    return trasnformed_df


def analyze_by_department(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza un análisis de la información por departamento.

    Args:
    - df: Tabla de Pandas con la información completa.

    Returns:
    - pd.DataFrame: Tabla con el análisis realizado por departamento.
    """
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")
    df["anio"] = df["Fecha"].dt.year

    df_by_dep = df.groupby(["dependencia", "anio"]).agg({
                    'Sueldo Neto': ['sum', 'count', 'mean', 'min', 'max']
                }).reset_index()
    df_by_dep.columns = ['Tipo', 'anio', 'Suma_Total_sueldos',
                            'Conteo_Empleados', 'Promedio_sueldo',
                            'Salario_Minimo', 'Salario_Maximo']
    
    return df_by_dep


def create_boxplot(file_name: str, column: str,
    agg_fn: callable = pd.DataFrame.sum) -> None:
    """
    Crea un diagrama de caja a partir de valores separados por comas.

    Args:
    - file_name (str): El nombre del archivo.
    - column (str): La columna para la cual crear el diagrama de caja.
    - agg_fn (callable): La función de agregación a usar.
    """
    df = pd.read_csv(file_name)
    df_by_type = df.groupby([column, "Fecha"])[["Sueldo Neto"]] \
                    .aggregate(agg_fn)
    df_by_type.boxplot(by=column, figsize=(27, 18))
    plt.xticks(rotation=90)
    plt.savefig(f"img/boxplot_{column}.png")
    plt.close()


def plot_by_dep(df: pd.DataFrame, dep: str) -> None:
    """
    Crea gráficos de línea y de caja para una dependencia específica.

    Args:
    - df (pd.DataFrame): La tabla con los datos.
    - dep (str): La dependencia para la cual crear los gráficos.
    """
    df_dep = df[df["dependencia"] == dep]

    df_dep.plot(y=["Sueldo Neto"], title=f"Línea de tiempo - {dep}")
    plt.savefig(f"img/lt_{dep}.png")
    plt.close()

    df_dep.boxplot(column=["Sueldo Neto"], by='dependencia')
    plt.savefig(f"img/bplt_{dep}.png")
    plt.close()


def create_plot_por_dependencia(file_name: str) -> None:
    """
    Crea gráficos de línea y de caja por dependencia a partir de valores
    separados por comas.

    Args:
    - file_name (str): El nombre del archivo.
    """
    df_complete = pd.read_csv(file_name)
    os.makedirs('img', exist_ok=True)
    df_by_dep = df_complete.groupby(["dependencia", "Fecha"])[["Sueldo Neto"]] \
                            .agg('count').reset_index()
    df_by_dep.set_index("Fecha", inplace=True)

    dependencias = df_by_dep["dependencia"].unique()
    for dep in dependencias:
       plot_by_dep(df_by_dep, dep)


    df_aux = df_complete.groupby(["Fecha", "dependencia"])[['Sueldo Neto']] \
                        .mean().unstack()
    df_aux.plot(y='Sueldo Neto', legend=False, figsize=(32, 18))
    plt.xticks(rotation=90)
    plt.savefig("img/foo.png")
    plt.close()

def anova(df_aux: pd.DataFrame, str_ols: str):
    """
    Realiza un análisis de varianza una tabla dada.

    Args:
    - df_aux (pd.DataFrame): La tabla en la cual realizar el análisis.
    - str_ols (str): La fórmula para el modelo.
    """
    model = ols(str_ols, data=df_aux).fit()
    anova_df = sm.stats.anova_lm(model, typ=2)

    if anova_df["PR(>F)"][0] < 0.05:
        print("Hay diferencias significativas")
        print(anova_df)

        # Prueba de Tukey
        tukey = pairwise_tukeyhsd(df_aux['GastoSalarios'], df_aux['Tipo'])
        print(tukey)
    else:
        print("No hay diferencias significativas")

def anova_1(file_name: str):
    """
    Realiza un análisis de varianza para valores separados por comas.

    Args:
    - file_name (str): El nombre del archivo.
    """
    df_complete = pd.read_csv(file_name)

    df_by_type = df_complete.groupby(["Tipo", "Fecha"])[["Sueldo Neto"]] \
                            .sum().reset_index()
    
    df_aux = df_by_type.rename(columns={"Sueldo Neto": "GastoSalarios"})
    
    tipos_interes = ["ADMIN", "CENTRO", "OTRO", "HOSPITAL", "PREPARATORIA"]
    df_aux = df_aux[df_aux["Tipo"].isin(tipos_interes)]

    print(df_aux.head())
    print(df_aux.columns)

    anova(df_aux, "GastoSalarios ~ Tipo")

def analysis(file_name: str) -> None:
    """
    Realiza el análisis de valores separados por comas y muestra los resultados.

    Args:
    - file_name (str): El nombre del archivo a analizar.
    """
    df_complete = pd.read_csv(file_name)
    df_complete.columns = df_complete.columns.str.strip()  # Eliminar espacios adicionales
    
    print("Columnas del DataFrame en analysis:", df_complete.columns)
    print("Primeras filas del DataFrame en analysis:")
    print(df_complete.head())
    
    if 'dependencia' not in df_complete.columns or 'Tipo' not in df_complete.columns:
        raise KeyError("Las columnas 'dependencia' y 'Tipo' deben estar presentes en el archivo CSV")
    
    display_dataframe(df_complete[["dependencia", "Tipo"]].drop_duplicates().head(150))
    
    if 'Fecha' not in df_complete.columns:
        raise KeyError("La columna 'Fecha' debe estar presente en el archivo CSV")

    df_by_dep = df_complete.groupby(["dependencia", "Fecha"])[["Sueldo Neto"]] \
                            .sum().reset_index()
    df_by_dep.set_index("Fecha", inplace=True)

    display_dataframe(df_by_dep.head(5))

    for dep in df_by_dep["dependencia"].unique():
       plot_by_dep(df_by_dep, dep)

    df_aux = df_complete.groupby(["Fecha","dependencia"])[['Sueldo Neto']] \
                        .mean().unstack()
    df_aux.plot(y='Sueldo Neto', legend=False, figsize=(32,18))
    plt.xticks(rotation=90)
    plt.savefig("img/foo.png")
    plt.close()

    df_by_dep.boxplot(by ='dependencia', figsize=(32,18))
    plt.xticks(rotation=90)
    plt.savefig("img/boxplot.png")# , bbox_inches='tight')
    plt.close()

    df_by_type = df_complete.groupby(["Tipo", "Fecha"])[["Sueldo Neto"]].aggregate(pd.DataFrame.sum)# .count()
    df_by_type.boxplot(by = 'Tipo', figsize=(18,9))
    plt.xticks(rotation=90)
    plt.savefig("img/boxplot_tipo.png")
    plt.close()

    df_aux = df_by_type.reset_index().rename(columns={"Sueldo Neto": "GastoSalarios"})
    print(df_aux.head())

    anova(df_aux, "GastoSalarios ~ Tipo")

def create_typed_df(filename: str) -> pd.DataFrame:
    """
    Crear una tabla con tipos a partir de valores separados por comas.

    Args:
    - filename (str): El nombre del archivo.

    Returns:
    - pd.DataFrame: La tabla con tipos.
    """
    df_complete = pd.read_csv(filename)
    return transform_into_typed_df(df_complete)

def show_type_of_department():
    """
    Muestra los tipos de dependencias.
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    display_dataframe(df_complete[['dependencia', 'Tipo']] \
        .drop_duplicates().head(150))

def show_salary_and_count_by_dependency_and_date():
    """
    Muestra el salario y el conteo por dependencia y fecha
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_complete.columns = df_complete.columns.str.strip()  # Eliminar espacios adicionales
    
    print("Columnas del DataFrame en show_salary_and_count_by_dependency_and_date:", df_complete.columns)
    
    df_by_type = df_complete.groupby(["dependencia", "Fecha"]).agg({
        'Sueldo Neto': ['sum', 'count', 'mean', 'max']
    })
    
    # Imprimir la estructura del DataFrame
    print("Estructura del DataFrame resultante del groupby y agg:", df_by_type.head())
    
    # Asignar nombres de columnas
    df_by_type.columns = ['Total_sueldos', 'Conteo_Empleado', 'Promedio_sueldo', 'Salario_Maximo']
    df_by_type.reset_index(inplace=True)
    df_by_type.set_index("Fecha", inplace=True)
    
    display_dataframe(df_by_type.head(150))


def show_data_by_type_and_date():
    """
    Muestra los datos por tipo y fecha
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_by_type = df_complete.groupby(["Tipo", "Fecha"])[["Sueldo Neto"]].sum()
    display_dataframe(df_by_type.head(150))

def show_salary_and_count_by_type_and_date():
    """
    Muestra el salario y el conteo por tipo y fecha.
    """
    df_complete = pd.read_csv("csv/typed_uanl.csv")
    df_complete.columns = df_complete.columns.str.strip()  # Eliminar espacios adicionales
    
    print("Columnas del DataFrame en show_salary_and_count_by_type_and_date:", df_complete.columns)
    
    df_by_type = df_complete.groupby(["Tipo", "Fecha"]).agg({
        'Sueldo Neto': ['sum', 'count', 'mean', 'max']
    })
    
    # Imprimir la estructura del DataFrame
    print("Estructura del DataFrame resultante del groupby y agg:", df_by_type.head())
    
    # Asignar nombres de columnas
    df_by_type.columns = ['Total_sueldos', 'Conteo_Empleado', 'Promedio_sueldo', 'Salario_Maximo']
    df_by_type.reset_index(inplace=True)
    df_by_type.set_index("Fecha", inplace=True)
    
    display_dataframe(df_by_type.head(150))

if __name__ == "__main__":
    typed_df = create_typed_df("csv/uanl.csv")
    display_dataframe(typed_df.head(50))
    typed_df.to_csv("csv/typed_uanl.csv", index=False)

    analyzed_df = analyze_by_department(typed_df)
    analyzed_df.to_csv("csv/analyzed_uanl.csv", index=False)

    show_type_of_department()

    show_data_by_type_and_date()

    show_salary_and_count_by_type_and_date()

    show_salary_and_count_by_dependency_and_date()

    analysis("csv/typed_uanl.csv")

    create_boxplot("csv/typed_uanl.csv", 'dependencia', pd.DataFrame.sum)#"Tipo")
    
    create_plot_por_dependencia("csv/typed_uanl.csv")
    
    anova_1("csv/typed_uanl.csv")


In [ ]:
def analysis(file_name: str) -> pd.DataFrame:
    df = pd.read_csv(file_name)
    df["hab_x_km2"] = df["poblacion_2020"] / df["area_km"]
    df["hab_x_mi"] = df["poblacion_2020"] / df["area_mi"]
    print(sum(df["poblacion_2020"]))
    return df

df = analysis("csv/estados_limpio.csv")
display_dataframe(df.head())
display_dataframe(df.describe())
print(df["poblacion_2020"].sum())